<h1>Using <i style="color:red;">SQLAlchemy</i>'s ORM in <i style="color:green;">Django</i></h1>

<h3><i>by Jack Camier, Python Developer</i></h3>

# Django is one of the most popular web frameworks in Python. As of this talk, 2027 companies reportedly use Django in their tech stacks, including YouTube, Bitbucket, Dropbox, Instagram, Pinterest, Udemy just to name a few...
https://stackshare.io/django

In [ ]:
%%HTML
<img src="django_logo.png" width="300"/>

https://www.djangoproject.com/

## Personally, I am a big fan and use it often. However, one aspect of Django that I don't like that much is its ORM. I am not alone and have heard from developers their frustrations with the ORM. It works great for simple projects and queries but whenever you try to do something more complex, you can run into problems.

## In this quick tutorial, I will show you a way you can use SQLAlchemy's ORM in conjunction with your Django Projects.

## We will use the Django tutorial as our baseline to see how this can work.
https://docs.djangoproject.com/en/3.0/intro/tutorial01/

## I am using the latest Django version as of this writing 3.0.5

In [1]:
import django

django.VERSION

(3, 0, 5, 'final', 0)

## Steps from the online tutorial, I had problems doing using the Jupyter Notebook. So these are the steps I did to get the project setup. django `mysite` is available in the git repo.

## created a virtualenv called `py37` with python 3.7.7

## You could do the same with `conda create --name py37 python=3.7`

## `django-admin startproject mysite`

## Go into the `mysite` directory and `python manage.py startapp polls`

## Create a URLconf in the `polls` directory, create a file called `urls.py`

## Next step is to point the root URLconf at the polls.urls module. In `mysite/urls.py`, add an import for django.urls.include and insert an include() in the urlpatterns list

## `python manage.py migrate`

## The migrate command looks at the INSTALLED_APPS setting and creates any necessary database tables according to the database settings in your `mysite/settings.py`

## `auth_user` is the key table that is created to manage users

## Create the models (Tables) in the `polls/models.py` module

## Add `'polls.apps.PollsConfig',` to the `mysite/settings.py` module. This is to allow Django to know that our app is in our project.

## The format is normally app_name.apps.App_nameConfig.

## So `polls` becomes `polls.apps.PollsConfig`

## It should look something like this now:

## Then, `python manage.py makemigrations polls` to update the migration

## Next, `python manage.py migrate` to build (create) the tables stored in migrations

# -------
## Up to this point, this has all been standard Django functionality. Now, this is where I will show a way you can use SQLAlchemy to working with Django

# -------

## Make sure you have SQLAlchemy installed
`pip install SQLAlchemy`
## I have included it in my requirements.txt file already.